In [45]:
import os
import pandas as pd
import fitz ##PyMuPDF
import glob
import copy
import os
from pprint import pprint

In [46]:
## ラストストーンをどちらが持っているか?
def solve_laststone(df):
    for i , r in df.iterrows():
        if i == 0:
            continue
        target_value = 0.0
        s = df.loc[df.index == i - 1]    
        if ( (s.red.values[0] == 0) & (s.yellow.values[0] == 0) ): ## ZeroZeroならば、動かない
            target_value = s.last_stone_is_red.values[0]
        else: ## 得点が入れば、点を失ったもの後攻＝　赤の点数が黄色より大きい場合　黄色が後攻
            if (s.red.values[0] < s.yellow.values[0] ):
                target_value = True
            else:
                target_value = False
        df.loc[df.index == i,'last_stone_is_red']= target_value
    return df

def pageToDataFrame(page,num):
    tabs = page.find_tables()
    if len(tabs.tables) <= 0: return None
    ## 1つ目のtableを作る = > Hanmerがわかる表を探索
    def GetScore( tabs ):
        for qtable in tabs.tables:
            scoreTable = qtable.extract()
            df = pd.DataFrame(scoreTable)
            if False == (len(df) == 2) :return False  , None 
            is_redHammer = (df[0].values == ['*','']).all()
            is_YellowHammer = (df[0].values == ['','*']).all()
            ##print(df)
            if any([is_redHammer,is_YellowHammer]):
                return True , df
            pass
        return False  , None
    isFind , df = GetScore(tabs)
    if False == isFind:
        print('No Score',num )
        return None
    is_redHammer = (df[0].values == ['*','']).all()
    is_YellowHammer = (df[0].values == ['','*']).all()

    df = df.drop(0,axis=1)
    colname = df.columns.to_list()
    colname = list(map( str , colname))
    colname = colname[:-1] + ['Total']
    df.columns = colname
    df = df.T
    df = df.reset_index()
    df.columns = ['ends','red','yellow']

    def to_number(v):
        if v.isdecimal():return int(v)
        else:return None
    df['red'] = df['red'].map( to_number )
    df['yellow'] = df['yellow'].map( to_number )
    #df = df.dropna(subset=['red','yellow'],axis=0)
    df = df.astype( {'red':float,'yellow':float})

    df['red_diff'] = df['red']
    df.loc[df['yellow'] > 0,'red_diff'] = df.loc[df['yellow'] > 0,'yellow'] * -1
    ## 残end数
    index_v = df.index.to_list()
    df['remain_ends'] = [8-int(v) for v in index_v[:-1]] + [0]
    
    df.loc[df['remain_ends'] < 0 , 'remain_ends'] = 0
    assert(any([is_redHammer,is_YellowHammer]))
    df['last_stone_is_red'] = False
    df.loc[ df.index == 0,'last_stone_is_red' ] = is_redHammer
    df = solve_laststone(df)
    df['page'] = num
    return df


In [17]:
def keyPosion( test_array , key ):
    if not key in test_array: return -1
    else: return test_array.index(key)

In [47]:
outpath = './img/'
def SaveImage( doc ,outpath , game , ends , maxpage ):
    if ends == 'Total' : 
        return pd.Series(["" , "This is Total"]) 
    ends = int(ends)
    taget_page = game+ends
    if maxpage <= taget_page:
        return pd.Series(["" , f'Over Page:{taget_page}'])
    page = doc[taget_page]
    text = page.get_text()
    text_array = text.split('\n')
    index = keyPosion(text_array,'Game - Shot by Shot')
    #print(index, index < 0)
    if index < 0:
        err = f'{game},{ends}:game error : no Shot by Shot'
        #print(err)
        return pd.Series(["" , err]) 
    assert( index > 0 )
    for text in text_array:
        if 'End ' in text:
            page_ends = int(text.split(' ')[-1])
            #print(page_ends,ends,text.split(' ')[-1] )
            if (page_ends == ends):
                pass
            else:
                err = f'{game},{ends}:ends error : no Shot by Shot'
                #print(err)
                return pd.Series(["" , err]) 
            break
    
    cimageList = page.get_images()
    dq = pd.DataFrame(cimageList)
    dq.to_csv('imgList.csv')
    def IndexFinder(dq):
        dqz = dq.loc[ (dq[5] == 'Indexed') & (dq[3] > 550) & (dq[3] < 610)].copy()
        dqz['sort'] = dq[7].map(lambda s:int(s[2:]))
        if False == (len(dqz) == 16): return -1 , 'NG'
        dqz = dqz.sort_values('sort')
        ##print(dqz)
        target_s = dqz.iloc[14]
        return target_s.iloc[0] , target_s.iloc[7]
    imageId , name = IndexFinder(dq)
    ##print(imageId , type(imageId) , name )
    if imageId < 0:
        err = f'{game}-{ends}:no image'
        #print(err)
        return pd.Series(["" , err]) 
    imageId = int(imageId)
    img = doc.extract_image(imageId)
    ext = img['ext']
    path = os.path.join(outpath,f'geme{game}end{ends}.{ext}')
    with open(path, "wb") as ofh:
        ofh.write(img['image'])
    return pd.Series([os.path.basename(path) ,'OK'])

# ここから

In [36]:
pdf_dir = '../pdf/*.pdf'
g = glob.glob( pdf_dir )

In [37]:
outdir='./dataset'
os.path.exists(outdir)

True

In [39]:
for fn in g:
    print('start:',fn)
    q = os.path.basename(fn).split('.')[0]
    path = os.path.join(outdir,q)
    os.makedirs(path,exist_ok=True)
    doc = fitz.open(fn) 
    page_list = []
    for i ,page in enumerate(doc):   
        if i < 5 : continue
        text = page.get_text()
        q = text.split('\n')
        key = 'Game Results'  
        title_index = keyPosion( q , key )
        if title_index < 0 :pass
        else:
            page_list.append(i)
    print(f'max page { i } ')
    maxpage = i
    l = []
    for i in page_list:
        l.append( pageToDataFrame( doc[i], i )  )
    dq = pd.concat(l)
    dq[['filepath','INFO']] = dq.apply(lambda r: SaveImage( doc, path , r.page , r.ends ,maxpage) , axis=1)
    csvpath = path+'.csv'
    dq.to_csv(csvpath)
    print('fin:',fn)
    

start: ../pdf\CWC2018-19_Leg1_ResultsBook.pdf
max page 681 
fin: ../pdf\CWC2018-19_Leg1_ResultsBook.pdf
start: ../pdf\CWC2018-19_Leg2_ResultsBook.pdf
max page 317 
fin: ../pdf\CWC2018-19_Leg2_ResultsBook.pdf
start: ../pdf\CWC2018-19_Leg3_ResultsBook.pdf
max page 750 
fin: ../pdf\CWC2018-19_Leg3_ResultsBook.pdf
start: ../pdf\ECC2017_ResultsBook_Men_A-Division.pdf
max page 540 
fin: ../pdf\ECC2017_ResultsBook_Men_A-Division.pdf
start: ../pdf\ECC2017_ResultsBook_Women_A-Division.pdf
max page 556 
fin: ../pdf\ECC2017_ResultsBook_Women_A-Division.pdf
start: ../pdf\ECC2018_ResultsBook_Men_A-Division.pdf
max page 543 
fin: ../pdf\ECC2018_ResultsBook_Men_A-Division.pdf
start: ../pdf\ECC2018_ResultsBook_Women_A-Division.pdf
max page 555 
fin: ../pdf\ECC2018_ResultsBook_Women_A-Division.pdf
start: ../pdf\ECC2019_ResultsBook_Men_A-Division.pdf
max page 542 
fin: ../pdf\ECC2019_ResultsBook_Men_A-Division.pdf
start: ../pdf\ECC2019_ResultsBook_Women_A-Division.pdf
max page 538 
fin: ../pdf\ECC2019_R

ValueError: operands could not be broadcast together with shapes (3,) (2,) 

In [44]:
g[16]

'../pdf\\OWG2018_ResultsBook.pdf'

In [48]:
for fn in g[16:]:
    print('start:',fn)
    q = os.path.basename(fn).split('.')[0]
    path = os.path.join(outdir,q)
    os.makedirs(path,exist_ok=True)
    doc = fitz.open(fn) 
    page_list = []
    for i ,page in enumerate(doc):   
        if i < 5 : continue
        text = page.get_text()
        q = text.split('\n')
        key = 'Game Results'  
        title_index = keyPosion( q , key )
        if title_index < 0 :pass
        else:
            page_list.append(i)
    print(f'max page { i } ')
    maxpage = i
    l = []
    for i in page_list:
        l.append( pageToDataFrame( doc[i], i )  )
    dq = pd.concat(l)
    dq[['filepath','INFO']] = dq.apply(lambda r: SaveImage( doc, path , r.page , r.ends ,maxpage) , axis=1)
    csvpath = path+'.csv'
    dq.to_csv(csvpath)
    print('fin:',fn)

start: ../pdf\OWG2018_ResultsBook.pdf
max page 1542 
No Score 1043
fin: ../pdf\OWG2018_ResultsBook.pdf
start: ../pdf\OWG2022_ResultsBook.pdf
max page 1622 
fin: ../pdf\OWG2022_ResultsBook.pdf
start: ../pdf\PACC2017_ResultsBook_Men.pdf
max page 128 
fin: ../pdf\PACC2017_ResultsBook_Men.pdf
start: ../pdf\PACC2017_ResultsBook_Women.pdf
max page 127 
fin: ../pdf\PACC2017_ResultsBook_Women.pdf
start: ../pdf\PACC2018_ResultsBook_Men.pdf
max page 441 
fin: ../pdf\PACC2018_ResultsBook_Men.pdf
start: ../pdf\PACC2018_ResultsBook_Women.pdf
max page 276 
fin: ../pdf\PACC2018_ResultsBook_Women.pdf
start: ../pdf\PACC2019_ResultsBook_Men.pdf
max page 145 
fin: ../pdf\PACC2019_ResultsBook_Men.pdf
start: ../pdf\PACC2019_ResultsBook_Women.pdf
max page 135 
fin: ../pdf\PACC2019_ResultsBook_Women.pdf
start: ../pdf\PACC2021_ResultsBook_Men.pdf
max page 50 
fin: ../pdf\PACC2021_ResultsBook_Men.pdf
start: ../pdf\PCCC2022_ResultsBook_Men_A-Division.pdf
max page 332 
fin: ../pdf\PCCC2022_ResultsBook_Men_A-Divi